In [2]:

!pip install -q --upgrade awscli boto3
# !pip install -q --upgrade PyYAML==6.0
!pip install -q --upgrade sagemaker==2.165.0
!pip install pip
!pip install scikit-learn==1.3.0
!pip install pandas==1.3.3
!pip install joblib
!pip install matplotlib==3.6.0


You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
  Using cached scikit_learn-1.3.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.1 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
  Using cached pandas-1.3.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Attem

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import pandas as pd

from pathlib import Path
import sagemaker
from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.sklearn.processing import SKLearnProcessor

from sagemaker.inputs import FileSystemInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.parameters import ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo, ConditionLessThan
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.fail_step import FailStep



role = sagemaker.get_execution_role()

'arn:aws:iam::253909639528:role/service-role/AmazonSageMaker-ExecutionRole-20230826T112945'

In [5]:
BUCKET = 'vmate-mnist-1'

!aws s3api create-bucket --bucket $BUCKET --create-bucket-configuration LocationConstraint="eu-north-1"


An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


Let's load the first 10 rows of the test set.

In [7]:
df = pd.read_csv(DATASET_FOLDER / 'mnist_train.csv', nrows=10)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
S3_LOCATION = f's3://{BUCKET}/mnist'

TRAIN_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(DATASET_FOLDER / 'mnist_train.csv'), 
    desired_s3_uri=S3_LOCATION,
)

TEST_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(DATASET_FOLDER / 'mnist_test.csv'), 
    desired_s3_uri=S3_LOCATION,
)

PROCESSED_SET_S3_BASE_URI = f's3://{BUCKET}/mnist/preprocessed_data'


print(f'Train set S3 location: {TRAIN_SET_S3_URI}')
print(f'Test set S3 location: {TEST_SET_S3_URI}')
print(f'Processed set S3 location: {PROCESSED_SET_S3_BASE_URI}')


Train set S3 location: s3://vmate-mnist-1/mnist/mnist_train.csv
Test set S3 location: s3://vmate-mnist-1/mnist/mnist_test.csv
Processed set S3 location: s3://vmate-mnist-1/mnist/preprocessed_data


In [9]:
%%writefile {CODE_FOLDER}/preprocessor.py

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from pickle import dump
from typing import Tuple
import argparse

DEFAULT_BASE_DIR = Path('/opt')/'ml'/'processing'

def _preprocess_pipeline(df_data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:    
    num_classes = 10

    categorical_transformer = Pipeline(
        steps=[
            ('encoder', OneHotEncoder())            
        ]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ('labels', categorical_transformer, ['label'])
        ],
        remainder='passthrough'
    )

    pipeline = Pipeline(
        steps=[
            ("preprocess", preprocessor)
        ]
    )
    data: np.ndarray = pipeline.fit_transform(df_data)
    # OneHotEncoded
    y_data: np.ndarray = data[:, :num_classes]
    # Drop OneHotEncoded target variable
    data = np.delete(data,np.arange(num_classes), axis=1)

    X_train, X_test_validation, y_data, y_test_validation = train_test_split(data, y_data, test_size=0.2, random_state=7)
    X_test, X_validation, y_test, y_validation = train_test_split(X_test_validation, y_test_validation, test_size=0.5, random_state=7)

    return X_train, X_test, X_validation, y_data, y_test, y_validation


def _save_pipeline(base_dir: str, pipeline: Pipeline):  
    
    pipeline_path = Path(base_dir)
    pipeline_path.mkdir(parents=True, exist_ok=True)
    dump(pipeline, open(pipeline_path / 'pipeline.pkl', 'wb'))

def preprocess(base_dir = DEFAULT_BASE_DIR, data_dir=DEFAULT_BASE_DIR):
            
    base_dir = Path(base_dir)
    data_dir = Path(data_dir)
        
    (base_dir / 'train').mkdir(parents=True, exist_ok=True)
    (base_dir / 'validation').mkdir(parents=True, exist_ok=True)
    (base_dir / 'test').mkdir(parents=True, exist_ok=True)
    (base_dir / 'labels').mkdir(parents=True, exist_ok=True)
        
    df_data: pd.DataFrame =  pd.read_csv(Path(data_dir) / 'mnist_train.csv')

    df_data: pd.Dataframe = df_data.sample(frac=0.5, random_state=1)

    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(df_data['label'])

    X_train, X_test, X_validation, y_train, y_test, y_validation  = _preprocess_pipeline(df_data)    

    np.savetxt(base_dir / 'train' / 'mnist_train.csv', np.concatenate([X_train, y_train], axis=1), delimiter=',')
    np.savetxt(base_dir / 'test' / 'mnist_test.csv', np.concatenate([X_test, y_test], axis=1), delimiter=',')
    np.savetxt(base_dir / 'validation' / 'mnist_validation.csv', np.concatenate([X_validation, y_validation], axis=1), delimiter=',')
    np.savetxt(base_dir / 'labels' / 'labels.csv', labels, delimiter=',')

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--base_dir', type=str, dest='base_dir', default=DEFAULT_BASE_DIR)
    parser.add_argument('--data_dir', type=str, dest='data_dir', default="/opt/ml/processing/input")

    args, _ = parser.parse_known_args()
    
    preprocess(
        base_dir=args.base_dir,
        data_dir=args.data_dir
    )

Overwriting code/preprocessor.py


In [10]:
%%writefile {CODE_FOLDER}/train.py

from pathlib import Path
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
from joblib import Memory
from typing import Tuple
import argparse
import os

image_size = 28 * 28

def load_csv(file_path,  sample=None) -> Tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray]:
    num_classes = 10
    
    df = pd.read_csv(file_path)
    df_X_train = df.sample(frac=sample, random_state=1)
    df_X_test = df.sample(frac=sample, random_state=1)

    y_train = df_X_train.values[:, -num_classes:]
    y_test = df_X_test.values[:, -num_classes:]
    
    
    return df_X_train.values, y_train, df_X_test.values, y_test

def create(no_features):  
    model = Sequential([
        Dense(32, activation='sigmoid'),
        Dense(10, activation='softmax'),
    ])
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    model.build(input_shape=(None,no_features)) # Comeback to this, find out what 'None' is.  
    return model

def train(train_dir,
          model_dir,
          epochs=10,
          batch_size=16):
    
    X_train, y_train, X_test, y_test = load_csv(Path(train_dir) / "mnist_train.csv", 
                                                0.5)
    
    model = create(X_train.shape[1])
    model.summary()
    
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1, verbose=True)
    model.save(Path(model_dir) / "001")
    loss, accuracy = model.evaluate(X_test, y_test, verbose=True)

    print(f'MODEL_DIRECTORY: {model_dir}')
    print(f'Loss: {loss:.3}, Accuracy: {accuracy:.3}')
    
if __name__ == "__main__":
    # https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_dir", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--model_dir", type=str, default='/opt/ml/model')

    args, _ = parser.parse_known_args()
    train(
        train_dir=args.train_dir,
        model_dir="/opt/ml/model",
    )
        


Overwriting code/train.py


In [11]:
%%writefile {CODE_FOLDER}/evaluation.py

import os
import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score

from typing import Tuple
# from train import load_csv

from sklearn.preprocessing import OneHotEncoder, LabelEncoder


MODEL_PATH = "/opt/ml/processing/model"
TEST_PATH = "/opt/ml/processing/test"
OUTPUT_PATH = "/opt/ml/processing/evaluation"

def load_csv(file_path,  sample=None) -> Tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray]:
    num_classes = 10
    
    df = pd.read_csv(file_path)
    df_X_train = df.sample(frac=sample, random_state=1)
    df_X_test = df.sample(frac=sample, random_state=1)

    y_train = df_X_train.values[:, -num_classes:]
    y_test = df_X_test.values[:, -num_classes:]
    
    return df_X_train.values, y_train, df_X_test.values, y_test

def evaluate(model_path, test_path, output_path):
    # The first step is to extract the model package so we can load 
    # it in memory.
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    model = keras.models.load_model(Path(model_path) / "001")
    
    X_train, y_train, X_test, y_test = load_csv(Path(test_path) / "mnist_test.csv", 0.5)

    
    predictions = np.argmax(model.predict(X_test), axis=-1)

    encoder = OneHotEncoder()
    predictions = encoder.fit_transform(predictions.reshape(-1,1)) # Reshape to a 2-D array
    accuracy = accuracy_score(y_test, predictions)
    print(f"Test accuracy: {accuracy}")

    # Let's create an evaluation report using the model accuracy.
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": accuracy
            },
        },
    }
    
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))


if __name__ == "__main__":
    evaluate(
        model_path=MODEL_PATH, 
        test_path=TEST_PATH,
        output_path=OUTPUT_PATH
    )

Overwriting code/evaluation.py


In [12]:
from preprocessor import preprocess
from train import train
from evaluation import evaluate
import tempfile, tarfile
from pathlib import Path
import os

with tempfile.TemporaryDirectory() as directory:

    preprocess(
        base_dir=f"{directory}/processing",
        data_dir='./dataset'
    )

    train(
        train_dir=f"{directory}/processing/train",
        model_dir=Path(directory) / "model",
        epochs=1
    )
    
    with tarfile.open(Path(directory) / "model" / "model.tar.gz", "w:gz") as tar:
        tar.add(Path(directory) / "model" / "001", arcname="001")

    evaluate(
        model_path=Path(directory) / "model", 
        test_path=Path(directory) / "processing" / "test",
        output_path=Path(directory) / "evaluation"
    )

['processing']
processing
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                25440     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 25,770
Trainable params: 25,770
Non-trainable params: 0
_________________________________________________________________
Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2023-09-04 09:27:32.310 tensorflow-2-6-cpu-py-ml-t3-medium-97b05bbe92bff9d43abb12500182:52 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-09-04 09:

INFO:tensorflow:Assets written to: /tmp/tmpid5r4y22/model/001/assets


375/375 [==============================] - 2s 4ms/step - loss: 1.2792 - accuracy: 0.6998
MODEL_DIRECTORY: /tmp/tmpid5r4y22/model
Loss: 1.28, Accuracy: 0.7
Test accuracy: 0.6913333333333334


In [13]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.parameters import ParameterString

train_input= ParameterString(
    name='train_data_location',
    default_value=TRAIN_SET_S3_URI
)

test_input= ParameterString(
    name='test_data_location',
    default_value=TEST_SET_S3_URI
)

processed_data_output_location= ParameterString(
    name='processed_data_output_location',
    default_value=PROCESSED_SET_S3_BASE_URI
)

In [14]:
cache_config = CacheConfig(
    enable_caching=True,
    expire_after='15d',
)

sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type="ml.t3.medium",
    instance_count=1,
)

preprocessing_step = ProcessingStep(
    name='preprocessing',
    processor = sklearn_processor,
    inputs=[
        ProcessingInput(source=train_input, destination='/opt/ml/processing/input'),
    ],
    outputs = [
        ProcessingOutput(output_name='train', source='/opt/ml/processing/train', destination=processed_data_output_location),
        ProcessingOutput(output_name='validation', source='/opt/ml/processing/validation', destination=processed_data_output_location),
        ProcessingOutput(output_name='test', source='/opt/ml/processing/test', destination=processed_data_output_location),
    ],
    code='code/preprocessor.py',
    cache_config=cache_config,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [15]:
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.tensorflow import TensorFlow
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.functions import Join
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.workflow.model_step import ModelStep

tensorflow_processor = TensorFlowProcessor(
    framework_version='2.6',
    role=role,
    instance_type='ml.t3.medium',
    instance_count=1,
    py_version='py38',
    sagemaker_session=PipelineSession()    
)

# This is a workaround to a problem with the SageMaker SDK: 
# By default, the TensorFlowProcessor runs the script using
# /bin/bash as its entrypoint. We want to ensure we run it 
# using python3.
tensorflow_processor.framework_entrypoint_command = ["python3"]


estimator = TensorFlow(
    entry_point=f"{CODE_FOLDER}/train.py",    
    framework_version="2.6",
    instance_type="ml.m5.large",
    py_version="py38",
    instance_count=1,
    script_mode=True,
    
    # The default profiler rule includes a timestamp which will change each time
    # the pipeline is upserted, causing cache misses. Since we don't need
    # profiling, we can disable it to take advantage of caching.
    disable_profiler=True,

    role=role,
    sagemaker_session=PipelineSession(),
)

step_args = estimator.fit(
    inputs={
        'train': TrainingInput(
            s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                'train'
            ].S3Output.S3Uri,
            content_type='text/csv'
        ),
        'validation': TrainingInput(
            s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                'validation'
            ].S3Output.S3Uri,
            content_type='text/csv'
        ),
        'test': TrainingInput(
            s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                'test'
            ].S3Output.S3Uri,
            content_type='text/csv'
        ),
    }
)

training_step = TrainingStep(
    name='train',
    step_args=step_args,
    cache_config=cache_config,
)

evaluation_report = PropertyFile(
    name="evaluation-report",
    output_name="evaluation",
    path="evaluation.json"
)

evaluation_step = ProcessingStep(
    processor=tensorflow_processor,
    name="evaluation",
    inputs=[
        ProcessingInput(source=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
            "test"
        ].S3Output.S3Uri,
        destination="/opt/ml/processing/test"
    ),
    ProcessingInput(
        source=(
            training_step.properties.ModelArtifacts.S3ModelArtifacts
        ),
        destination="/opt/ml/processing/model",
    )],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=f"{S3_LOCATION}/evaluation"),
    ],
    code=f"{CODE_FOLDER}/evaluation.py",
    cache_config=cache_config,
    property_files=[evaluation_report],
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="/", values=[
            evaluation_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'],"evaluation.json"]
        ),
        content_type="application/json",
    )
)

model_package_group_name = "mnist"
model = TensorFlowModel(
    model_data=(
        training_step.properties.ModelArtifacts.S3ModelArtifacts
    ),
    framework_version="2.6",
    sagemaker_session=PipelineSession(),
    role=role,
)

def create_model_registry_args(model, model_package_group_name, model_metrics, approval_status="PendingManualApproval"):
    return model.register(
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=approval_status,      
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        transform_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
    )

register_args_approval = create_model_registry_args(model, 
                                                    model_package_group_name, 
                                                    model_metrics, 
                                                    "Approved")

register_args_pending = create_model_registry_args(model,
                                                model_package_group_name, 
                                                model_metrics)

register_model_step_approved = ModelStep(
    name="register-model-approved_step",
    step_args=register_args_approval,
)

register_model_step_pending = ModelStep(
    name="register-model-pending",
    step_args=register_args_pending,
)

accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.70
)

min_accuracy_threshold = ParameterFloat(
    name="accuracy_threshold_min", 
    default_value=0.50
)

condition_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold
)

condition_lt = ConditionLessThan(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=min_accuracy_threshold
)

fail_step_min = FailStep(
    name="fail-min",
    error_message=Join(
        on=" ", 
        values=[
            "Execution failed because the model's accuracy was lower than", 
            min_accuracy_threshold
        ]
    ),
)

model_registry_approved = ConditionStep(
    name="register-model-approved",
    conditions=[condition_gte],
    if_steps=[register_model_step_approved],
    else_steps=[fail_step_min]
)


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:286: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [16]:

pipeline = Pipeline(
    name="mnist",
    parameters=[
        train_input,
        test_input,
        processed_data_output_location,
        accuracy_threshold,
        min_accuracy_threshold,
    ],
    steps=[preprocessing_step,
           training_step,
           evaluation_step,
           model_registry_approved,
    ]
)

pipeline.upsert(role_arn=role)
pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


_PipelineExecution(arn='arn:aws:sagemaker:eu-north-1:253909639528:pipeline/mnist/execution/d4ivvlrxjhhu', sagemaker_session=<sagemaker.session.Session object at 0x7f9c30e05730>)